## 需要安装贝叶斯模型：bayes_opt
#### 打开Anaconda Navigator--Enviroments--ml37--Open Terminal  
#### 在命令行模式下输入： pip install bayesian-optimization

In [1]:
from bayes_opt import BayesianOptimization
print("Test Bayesian Optimization OK")

Test Bayesian Optimization OK


In [2]:
#以下测试贝叶斯优化，求出最速降曲线
import tensorflow as tf
import numpy as np
from bayes_opt import BayesianOptimization
from scipy.interpolate import make_interp_spline
import matplotlib.pyplot as plt
import math
import os
#from sklearn.preprocessing import StandardScaler #在本工作中9根柱子的参数范围接近，不需要数据标准化
from keras.models import load_model

#调用之前NN训练好的模型，根据9根柱子高度，即时预测下降时间
my_model = load_model('time.h5')

CurveNum=10 #从前期数据中，找出50条下降最快的曲线
PoleNum=9  #使用多少根柱子来确定曲线形状
LowPole=[[0 for i in range(PoleNum)] for i in range(CurveNum)] #存储多根下降时间最短的曲线所对应的9个柱子高度
LowTotalTime=[0]*CurveNum   #存储多个下降最快的时间值

height=1000.0   #坡的高度为1000m
endheight=height*0.4 #确定纵轴终点，最好是略高于0点
width=800.0     #坡的宽度为800m
PoleSize=2000 # 相邻柱子之间考虑1000个点
wsize=PoleSize*(PoleNum+1)  #拟合曲线一共考虑2000*（9+1）=20000个点
wgrid=width/wsize    #x轴（坡的宽度方向）的格点分辨率
x_smooth=[0]*(wsize+1)   #定义20000+1个点，记录x值
for i in range(wsize+1):
    x_smooth[i]=i*wgrid
    
Pgrid=width/(PoleNum+1)   #每根柱子的格点分辨率
x=[0]*(PoleNum+2)    #定义9+2个点，记录起点、9根柱子、终点的x值（坡的宽度方向）
for i in range(PoleNum+2):
    x[i]=Pgrid*i

y=[0]*(PoleNum+2)    #定义9+2个点，记录起点、9根柱子、终点的y值（坡的高度方向）
y[0]=height #确定纵轴起点
y[PoleNum+1]=endheight #确定纵轴终点，最好是略高于0点


def Optfunc2(pole1, pole2, pole3, pole4, pole5, pole6, pole7, pole8, pole9):
    xval=np.zeros((1,9))  #生成二维数组
    xval[0]=[pole1, pole2, pole3, pole4, pole5, pole6, pole7, pole8, pole9]
    '''
    #数据标准化，在本工作中，由于9根柱子的参数范围都很接近，不需要进行数据标准化
    scaler = StandardScaler()  #定义标准化参数
    xval = scaler.fit_transform(xval) #将输入表格的描述符dataframe给标准化
    '''
    pred = my_model.predict(xval)  #调用model.predict能力，利用描述符来预测结果
    return -pred[0][0]




range0 = [[1, 999], [1, 999], [1, 999], [1, 999], [1, 999], [1, 999], [1, 999], [1, 999], [1, 999]]
print("统计速降曲线的柱子高度范围为：")
print(range0)
# 设定贝叶斯优化函数和取值范围
BO1 = BayesianOptimization(Optfunc2,
                           {'pole1': (range0[0][0], range0[0][1]),
                            'pole2': (range0[1][0], range0[1][1]),
                            'pole3': (range0[2][0], range0[2][1]),
                            'pole4': (range0[3][0], range0[3][1]),
                            'pole5': (range0[4][0], range0[4][1]),
                            'pole6': (range0[5][0], range0[5][1]),
                            'pole7': (range0[6][0], range0[6][1]),
                            'pole8': (range0[7][0], range0[7][1]),
                            'pole9': (range0[8][0], range0[8][1]),
                            })
# 进行贝叶斯优化，设定最初随机取值的次数，然后贝叶斯优化的次数，和采集函数
print("=========================================")
print("以下执行贝叶斯优化：")
BO1.maximize(40, 500, acq='ucb', kappa=0.3) #调用maximize函数寻找最优参数
#init_points=40：随机产生初始化点的个数为40个，值越大，找到起步点的效果可能更好
#n_iter=1200：进行贝叶斯优化的迭代次数，值越大，精度越高
#acq='ucb'：采集函数模型,【取值为'ucb','pi','ei'】
#kappa=0：ucb模型中的调节参数，越低越倾向于在当前最佳结果附近搜索（缺点：容易陷入局部最优）
#速降曲线，因为其解析解确定是摆线公式，不存在局部最优解，因此选择kappa=0.3
    
# 输出结果中的最大值
print(BO1.max)
print("优化结束")
print("=========================================")

# plz为调取值所用的列表，plx，ply为优化后的柱子坐标用于样条函数拟合
plz = ['pole1', 'pole2', 'pole3', 'pole4', 'pole5', 'pole6', 'pole7', 'pole8', 'pole9']
ply = [0] * 11
ply[0] = height
ply[10] = endheight
for i in range(PoleNum):
    ply[i + 1] = BO1.max['params'][plz[i]]
plx = [0] * (PoleNum+2)
for i in range(PoleNum+2):
    plx[i] = width/(PoleNum+1) * i
l1 = make_interp_spline(plx, ply)
lpoints = 2000  # 绘制优化后样条函数和摆线的细分数

# x，y为样条函数的细分坐标
x = [0] * lpoints
y = [0.0] * lpoints
for i in range(lpoints):
    x[i] = (width / lpoints) * i
y = l1(x)


plt.plot(x, y, label='optimization') # 绘制最优曲线进行比较 
plt.legend(['optimized'])
plt.show()

print("测试结束")

统计速降曲线的柱子高度范围为：
[[1, 999], [1, 999], [1, 999], [1, 999], [1, 999], [1, 999], [1, 999], [1, 999], [1, 999]]
以下执行贝叶斯优化：
|   iter    |  target   |   pole1   |   pole2   |   pole3   |   pole4   |   pole5   |   pole6   |   pole7   |   pole8   |   pole9   |
-------------------------------------------------------------------------------------------------------------------------------------


Exception: 
Passing acquisition function parameters or gaussian process parameters to maximize
is no longer supported. Instead,please use the "set_gp_params" method to set
 the gp params, and pass an instance of bayes_opt.util.UtilityFunction
 using the acquisition_function argument
